##Install required tools

In [ ]:
!pip install git+https://github.com/openai/whisper.git 
!pip install onnx
!pip install onnx_tf
!git clone https://github.com/usefulsensors/openai-whisper.git
!git clone https://github.com/openai/whisper.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-1j3vlkcm
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-1j3vlkcm
  Resolved https://github.com/openai/whisper.git to commit 12e1089462a2ea92e9ade6145e7be5b6883676ff
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'openai-whisper' already exists and is not an empty directory.
fatal: destination path 'whisper' already exists and is not an empty directory.


##Generate whisper encoder tflite(hybrid) model and run Inference 

In [ ]:
import whisper
import torch
import tensorflow as tf
import onnx
import numpy as np
import argparse
import os
import warnings
import tqdm
from onnx_tf.backend import prepare
from whisper.audio import load_audio, log_mel_spectrogram,pad_or_trim,N_FRAMES, SAMPLE_RATE
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

#load openai->whisper(pytorch)->tiny model
tiny_model = whisper.load_model("tiny")

#Export to onnx format
torch.onnx.export(tiny_model.encoder, torch.randn(1,80,3000).to(device), "./whisper-encoder.onnx")
onnx_model_path = './whisper-encoder.onnx'
tf_model_path = 'model_tf-encoder'

onnx_model = onnx.load(onnx_model_path)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)


saved_model_dir = 'model_tf-encoder'
tflite_model_path = 'whisper-encoder-hybrid.tflite'

def representative_dataset_random():
    for _ in range(100):
      data = np.random.rand(1, 80, 3000)
      yield [data.astype(np.float32)]

def representative_dataset():
    for _ in range(10):#Change this to 100 and provide 100 different audio files from known dataset 
      mel_from_file = log_mel_spectrogram('/content/whisper/tests/jfk.flac')
      segment = pad_or_trim(mel_from_file, N_FRAMES)
      segment = tf.expand_dims(segment, 0)
      print(segment.shape)
      yield [segment]

# Convert to tflite(int8) model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
#converter.representative_dataset = representative_dataset
#converter.inference_input_type = tf.int8  # or tf.uint8
#converter.inference_output_type = tf.int8  # or tf.uint8
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)
import tensorflow as tf
import numpy as np
tflite_model_path = '/content/whisper-encoder-hybrid.tflite'

# Load the TFLite model and allocate tensors
interpreter_enc = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter_enc.allocate_tensors()

print("== Input details ==")
print("name:", interpreter_enc.get_input_details()[0]['name'])
print("shape:", interpreter_enc.get_input_details()[0]['shape'])
print("type:", interpreter_enc.get_input_details()[0]['dtype'])

print("\nDUMP INPUT")
print(interpreter_enc.get_input_details()[0])

print("\n== Output details ==")
print("name:", interpreter_enc.get_output_details()[0]['name'])
print("shape:", interpreter_enc.get_output_details()[0]['shape'])
print("type:", interpreter_enc.get_output_details()[0]['dtype'])

print("\nDUMP OUTPUT")
print(interpreter_enc.get_output_details()[0])

# Get input and output tensors
input_details = interpreter_enc.get_input_details()
output_details = interpreter_enc.get_output_details()
output_tensor = interpreter_enc.get_output_details()[0]['index']

# Test the model with random data
input_shape = input_details[0]['shape']
mel_from_file = log_mel_spectrogram('/content/whisper/tests/jfk.flac')
input_tensor = pad_or_trim(mel_from_file, N_FRAMES)
input_tensor = tf.expand_dims(input_tensor, 0)

audio = whisper.load_audio('/content/whisper/tests/jfk.flac')
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio)
mel = np.expand_dims(mel,0)
#input_tensor = np.array(input_tensor-128, dtype=np.int8)
interpreter_enc.set_tensor(input_details[0]['index'], mel)

interpreter_enc.invoke()
print("Whisper Encoder Inference executed successfully\n")
encoder_output_data = interpreter_enc.get_tensor(output_tensor)
print(encoder_output_data.shape)
print(encoder_output_data)
np.savetxt("encoder_output.txt", encoder_output_data.reshape((3,-1)), fmt="%s", header=str(encoder_output_data.shape))

Using device: cpu


/usr/local/lib/python3.8/dist-packages/whisper/model.py:153: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert x.shape[1:] == self.positional_embedding.shape, "incorrect audio shape"


== Input details ==
name: serving_default_x.1:0
shape: [   1   80 3000]
type: <class 'numpy.float32'>

DUMP INPUT
{'name': 'serving_default_x.1:0', 'index': 0, 'shape': array([   1,   80, 3000], dtype=int32), 'shape_signature': array([   1,   80, 3000], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}

== Output details ==
name: PartitionedCall:0
shape: [   1 1500  384]
type: <class 'numpy.float32'>

DUMP OUTPUT
{'name': 'PartitionedCall:0', 'index': 557, 'shape': array([   1, 1500,  384], dtype=int32), 'shape_signature': array([   1, 1500,  384], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
Whisper Encoder Inference execute

## Convert from pytorch to decoder onnx

In [ ]:
import whisper
import torch

tiny_model = whisper.load_model("tiny")
#start_tokens = [50258, 50259, 50359, 50363] #<|startoftranscript|><|en|><|translate|><|notimestamps|>
torch.onnx.export(tiny_model.decoder, (torch.tensor([[50258, 50259, 50359, 50363]]).to(device), torch.randn(1, 1500, 384).to(device)), "/content/whisper-decoder-language.onnx")

##Generate decoder language tflite(hybrid) model and run Inference

In [ ]:
from onnx_tf.backend import prepare
import onnx

onnx_model_path = '/content/whisper-decoder-language.onnx'
tf_model_path = 'model_tf-decoder-language'

onnx_model = onnx.load(onnx_model_path)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)
saved_model_dir = 'model_tf-decoder-language'
tflite_model_path = 'whisper-decoder-language-hybrid.tflite'
#Change from random representative dataset to real representative dataset
def representative_dataset_random():
    for _ in range(1):
      input_tensor = np.random.rand(1,1500,384)
      decoder_input_ids = torch.tensor([[50258, 50259, 50359, 50363]])
      decoder_input_ids = np.array(decoder_input_ids, dtype=np.int64)
      yield [decoder_input_ids, input_tensor.astype(np.float32)]
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
#converter.representative_dataset = representative_dataset_random
#converter.inference_input_type = tf.int8  # or tf.uint8
#converter.inference_output_type = tf.int8  # or tf.uint8
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)
tflite_model_path = '/content/whisper-decoder-language-hybrid.tflite'



In [ ]:
import numpy as np
tflite_model_path='/content/whisper-decoder-language-int8.tflite'
#tflite_model_path='/content/whisper-decoder_main-int8.tflite'
print(tflite_model_path)

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

decoder_input_ids = torch.tensor([50258, 50259, 50359, 50363])
decoder_input_ids = tf.expand_dims(decoder_input_ids, 0)
print(decoder_input_ids.shape)
print(encoder_output_data.shape)

input_tensor_1 = interpreter.get_input_details()[0]['index']
interpreter.set_tensor(input_tensor_1, decoder_input_ids)

input_tensor_2 = interpreter.get_input_details()[1]['index']
interpreter.set_tensor(input_tensor_2, encoder_output_data)


print("== Input details ==")
print("name:", interpreter.get_input_details()[0]['name'])
print("shape:", interpreter.get_input_details()[0]['shape'])
print("type:", interpreter.get_input_details()[0]['dtype'])

print("\nDUMP INPUT")
print(interpreter.get_input_details()[0])

print("\n== Output details ==")
print("name:", interpreter.get_output_details()[0]['name'])
print("shape:", interpreter.get_output_details()[0]['shape'])
print("type:", interpreter.get_output_details()[0]['dtype'])

print("\nDUMP OUTPUT")
print(interpreter.get_output_details()[0])

interpreter.invoke()

output_data = interpreter.get_tensor(output_tensor)
print(output_data)
np.savetxt("output.txt", output_data.reshape((3,-1)), fmt="%s", header=str(output_data.shape))
output_details = interpreter.get_output_details()
print("%d outputs" % len(output_details))
for index, output_detail in enumerate(output_details):
  print("  Output %d" % index)
  print("    name:", output_detail['name'])
  print("    shape:", output_detail['shape'])
  print("    type:", output_detail['dtype'])

cleaned = np.argmax(output_data, axis=-1)
last_token = cleaned[0,-1]
print(last_token)



/content/whisper-decoder-language-int8.tflite
(1, 4)
(1, 1500, 384)
== Input details ==
name: serving_default_input.1:0
shape: [1 4]
type: <class 'numpy.int64'>

DUMP INPUT
{'name': 'serving_default_input.1:0', 'index': 0, 'shape': array([1, 4], dtype=int32), 'shape_signature': array([1, 4], dtype=int32), 'dtype': <class 'numpy.int64'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}

== Output details ==
name: StatefulPartitionedCall:0
shape: [    1     4 51865]
type: <class 'numpy.float32'>

DUMP OUTPUT
{'name': 'StatefulPartitionedCall:0', 'index': 939, 'shape': array([    1,     4, 51865], dtype=int32), 'shape_signature': array([    1,     4, 51865], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}

In [ ]:
%ls -la

total 270344
drwxr-xr-x  1 root root      4096 Jan 21 04:19 ./
drwxr-xr-x  1 root root      4096 Jan 21 03:30 ../
drwxr-xr-x  4 root root      4096 Jan 19 14:35 .config/
-rw-r--r--  1 root root   6354294 Jan 21 04:26 encoder_output.txt
drwxr-xr-x  2 root root      4096 Jan 21 04:18 .ipynb_checkpoints/
drwxr-xr-x  4 root root      4096 Jan 21 04:27 model_tf-decoder-language/
drwxr-xr-x  4 root root      4096 Jan 21 04:26 model_tf-encoder/
drwxr-xr-x 11 root root      4096 Jan 21 03:47 openai-whisper/
drwxr-xr-x  1 root root      4096 Jan 19 14:35 sample_data/
drwxr-xr-x  8 root root      4096 Jan 21 04:14 whisper/
-rw-r--r--  1 root root  30195320 Jan 21 04:27 whisper-decoder-language-int8.tflite
-rw-r--r--  1 root root 197262618 Jan 21 04:26 whisper-decoder-language.onnx
-rw-r--r--  1 root root  10099392 Jan 21 04:26 whisper-encoder-hybrid.tflite
-rw-r--r--  1 root root  32875460 Jan 21 04:26 whisper-encoder.onnx
